# Creating Monopoly Game Board Simulation with Python

---

## Import packages

In [1]:
# JSON manipulation
import json

# Randomization
import random

# Count unique values
from collections import Counter

# Matrices manipulation
import numpy as np

# Data frame manipulation
import pandas as pd

# Data viz
from plotnine import *
import plotnine

In [2]:
# Warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)

## Load the data

### The `chance` and `community chest`

In [7]:
# Open JSON file
card = open('../data/card/chance_community_card.json')
# Returns JSON object as a dictionary
data_card = json.load(card)

In [8]:
# Chance and community cards
data_card['chance_card']

[{'number': 1,
  'naration': 'Advance to Go (Collect Â£200)',
  'action': 'move_money',
  'asset': None,
  'money': {'amount': 200,
   'cash_in': True,
   'requisite': {'status': False, 'condition': None}},
  'move': {'properties': 1, 'forward': False, 'step': None}},
 {'number': 2,
  'naration': 'Advance to Trafalgar Square. If you pass Go, collect Â£200',
  'action': 'move_money',
  'asset': None,
  'money': {'amount': 200,
   'cash_in': True,
   'requisite': {'status': True, 'condition': 'pass_go'}},
  'move': {'properties': 25, 'forward': False, 'step': None}},
 {'number': 3,
  'naration': 'Advance to Mayfair',
  'action': 'move',
  'asset': None,
  'money': {'amount': 0,
   'cash': None,
   'requisite': {'status': False, 'condition': None}},
  'move': {'properties': 40, 'forward': False, 'step': None}},
 {'number': 4,
  'naration': 'Advance to Pall Mall. If you pass Go, collect Â£200',
  'action': 'move_money',
  'asset': None,
  'money': {'amount': 200,
   'cash_in': True,
   're

### The `spaces`

In [9]:
# Open JSON file
space = open('../data/asset/board_spaces.json')
# Returns JSON object as a dictionary
data_space = json.load(space)

In [10]:
# Spaces
data_space

[{'order': 1,
  'name': 'Go',
  'space_type': 'corner_square',
  'color': 'white',
  'money': {'amount': 200, 'cash_in': True}},
 {'order': 2,
  'name': 'Old Kent Road',
  'space_type': 'road',
  'color': 'brown',
  'money': {'amount': 60, 'cash_in': False},
  'rent': {'alone': 2,
   'monopoly': 4,
   'with_house': {'house_1': 10,
    'house_2': 30,
    'house_3': 90,
    'house_4': 160,
    'hotel': 250}},
  'prices': {'printed_price': 60, 'mortgage_value': 30, 'building_costs': 50}},
 {'order': 3,
  'name': 'Community Chest',
  'space_type': 'community_chest',
  'color': 'white',
  'money': {'amount': 0, 'cash_in': False}},
 {'order': 4,
  'name': 'Whitechapel Road',
  'space_type': 'road',
  'color': 'brown',
  'money': {'amount': 60, 'cash_in': False},
  'rent': {'alone': 4,
   'monopoly': 8,
   'with_house': {'house_1': 20,
    'house_2': 60,
    'house_3': 180,
    'house_4': 320,
    'hotel': 450}},
  'prices': {'printed_price': 60, 'mortgage_value': 30, 'building_costs': 50}},


## Take `chance` and `community chest` cards

### Functions for taking a card

In [11]:
# Update spaces
def update_space(
    last_move: int,
    dices: list,
    data_space: dict
    ):
    # Updated board based latest move
    l_board = (data_space[last_move - 1:] + data_space[:last_move - 1])
    # Current space
    d_current_space = [l_board[np.sum(dices) % len(l_board)]]
    
    return d_current_space

In [12]:
# Find the shortest spaces in circular list
def distance_space(
    data_space: dict,
    first_index: int,
    second_index: int
    ):
    # Update the data space
    data_space_update = (data_space[first_index - 1:] + data_space[:first_index - 1])
    
    # Calculate the distance
    distance = next((index for (index, dict_) in enumerate(data_space_update) if dict_['order'] == second_index), None)
    
    return distance

In [13]:
# Find the shortest spaces for spaces in circular list
def shortest_space(
    source: int,
    targets: list,
    data_space: dict
    ):
    # Create a dictionary
    d_targets = {key: None for key in targets}
    
    # Loop the targets
    for target in d_targets.keys():
        # Calculate the minimum distance
        shortest = distance_space(
            data_space = data_space,
            first_index = source,
            second_index = target
        )
        
        # Update the dictionary
        d_targets.update(
            {
                target: shortest
            }
        )
        
    # Get the best space
    space = min(d_targets, key = d_targets.get)

    # Forward-backward status
    forward_status = True
    
    return (space, forward_status)

In [14]:
# Select a card and show the result
def take_cards(
    subiteration_dict: dict,
    card: bool,
    chance_card: bool,
    data_card: dict,
    data_space: dict
    ):
    # Card status
    card_status = card
    
    # Card type
    card = 'community_card'

    # Chance or community chest card
    if chance_card:
        # Update the card type
        card = 'chance_card'
        # Select the card
        chance_cards = data_card['chance_card']
        selected_card = random.choice(chance_cards)
    else:
        # Select the card
        community_cards = data_card['community_card']
        selected_card = random.choice(community_cards)

    # CORE ALGORITHM
    # Update the card status
    subiteration_dict['move'][card] = selected_card['number']
    
    # Run the following command if card is used
    if card_status:
        # Move the space
        if selected_card['action'] in ['move', 'move_money']:
            # Last space
            last_space = subiteration_dict['move']['after']['real']
            if subiteration_dict['move']['after']['updated'] != None:
                last_space = subiteration_dict['move']['after']['updated']
            # Properties
            property_index = selected_card['move']['properties']

            # Move few step
            if selected_card['move']['properties'] == None:
                # Steps
                step = selected_card['move']['step']

                # Multiplier
                multiplier = -1
                # Update the multiplier
                if selected_card['move']['forward']:
                    multiplier = 1

                # Current space
                d_current_space_card = update_space(
                    last_move = last_space,
                    dices = (multiplier * step),
                    data_space = data_space
                )

                # Update the last space
                if card:
                    subiteration_dict['move']['after']['updated'] = d_current_space_card[0]['order']
                    subiteration_dict['move']['forward']['updated'] = selected_card['move']['forward']
                else:
                    pass

            # Move to the specific space
            else:
                # Go to nearest property (998: utilities, 999: stations)
                if property_index in [998, 999]:
                    # Get a list of spaces
                    targets = {
                        998: [13, 29],
                        999: [6, 16, 26, 36]
                    }
                    # Get the shortest space
                    update_index, forward_stat = shortest_space(
                        source = last_space,
                        targets = targets[property_index],
                        data_space = data_space
                    )
                    # Update the last space
                    if card:
                        subiteration_dict['move']['after']['updated'] = update_index
                        subiteration_dict['move']['forward']['updated'] = forward_stat
                    else:
                        pass

                # Go to specific property
                else:
                    # Update the last space
                    if card:
                        subiteration_dict['move']['after']['updated'] = property_index
                        subiteration_dict['move']['forward']['updated'] = True
                    else:
                        pass
    
    # Run the following command if card is not used
    else:
        pass
    
    return subiteration_dict

### Simulation

In [15]:
# Sample data
sample = {
    'subiteration': 1,
    'dices': [2, 3],
    'move': {
        'chance_card': None,
        'community_card': None,
        'jail': {
            'status': False,
            'number': 0
        },
        'before': 30,
        'after': {
            'real': 31,
            'updated': None
        },
        'forward': {
            'real': None,
            'updated': None
        }
    }
}

In [16]:
# Simulation
take_cards(
    subiteration_dict = sample,
    card = False,
    chance_card = True,
    data_card = data_card,
    data_space = data_space
)

{'subiteration': 1,
 'dices': [2, 3],
 'move': {'chance_card': 7,
  'community_card': None,
  'jail': {'status': False, 'number': 0},
  'before': 30,
  'after': {'real': 31, 'updated': None},
  'forward': {'real': None, 'updated': None}}}

## Players movement

### Functions for movement

In [17]:
# Rounds on Monopoly board
def round_spaces(
    dict_spaces: dict
    ):
    # Latest round
    latest_round = dict_spaces['round']
    
    # Update status
    updated_status = False
    # After-before spaces
    before = dict_spaces['move']['before']
    after = dict_spaces['move']['after']['real']
    if dict_spaces['move']['after']['updated'] != None:
        after = dict_spaces['move']['after']['updated']
        updated_status = True
    
    # Create a list
    l = list(range(before, after))
    # Dice roll
    dices_roll = abs(before - after)
    
    # Jail status
    jail_status = dict_spaces['move']['jail']['status']
    # Chance card
    chance_card = dict_spaces['move']['chance_card']
    # Community chest card
    community_card = dict_spaces['move']['community_card']
    
    # If player(s) is on a 'Jail'
    if jail_status == False:
        if (chance_card != None) and (community_card != None):
            # Check condition
            if len(l) == 0:
                # Move forward
                if dices_roll > 12:
                    dict_spaces['round'] += 1
                # Stay
                else:
                    pass
            else:
                # Move backward
                if dices_roll > 12:
                    dict_spaces['round'] -= 1
                # Stay
                else:
                    pass
        else:
            if (chance_card != None):
                pass
    else:
        pass
    
    return dict_spaces

In [18]:
# Player(s) movement
def move_spaces(
    dict_latest: dict,
    dices: int,
    subiteration: int,
    three_times: bool,
    card: bool,
    data_card: dict,
    data_space: dict
    ):
    # Template of players' iteration
    template_iter = {
        'subiteration': subiteration,
        'dices': dices,
        'move': {
            'chance_card': None,
            'community_card': None,
            'jail': {
                'status': False,
                'number': 0
            },
            'before': None,
            'after': {
                'real': None,
                'updated': None
            },
            'forward': {
                'real': None,
                'updated': None
            }
        }
    }
    
    # Jail status and number
    jail_status, jail_trial = dict_latest['move']['jail'].values()
    
    # latest space position
    last_move = dict_latest['move']['after']['real']
    if dict_latest['move']['after']['updated'] != None:
        last_move = dict_latest['move']['after']['updated']
    
    # If player(s) is on jail space
    if jail_status:
        if (len(np.unique(dices)) != 1):
            if (jail_trial < 2):
                # Update dictionary
                template_iter['move']['jail']['status'] = jail_status
                template_iter['move']['jail']['number'] = jail_trial + 1
                template_iter['move']['before'] = last_move
                template_iter['move']['after']['real'] = last_move
            else:
                # Current space
                d_current_space = update_space(
                    last_move = last_move,
                    dices = dices,
                    data_space = data_space
                )
                # Update dictionary
                template_iter['move']['before'] = last_move
                template_iter['move']['after']['real'] = d_current_space[0]['order']
                template_iter['move']['forward']['real'] = True
        else:
            # Current space
            d_current_space = update_space(
                last_move = last_move,
                dices = dices,
                data_space = data_space
            )
            # Update dictionary
            template_iter['move']['before'] = last_move
            template_iter['move']['after']['real'] = d_current_space[0]['order']
            template_iter['move']['forward']['real'] = True
    
    # If player(s) is not on jail space
    else:
        # If player(s) throw the same number three times
        if three_times:
            template_iter['move']['jail']['status'] = True
            template_iter['move']['after']['real'] = 11
            template_iter['move']['before'] = last_move
            template_iter['move']['forward']['real'] = False
        
        # Otherwise
        else:
            # List of space orders based on space type
            d_space = {
                'community_chest': [space['order'] for space in data_space if space['space_type'] == 'community_chest'],
                'chance': [space['order'] for space in data_space if space['space_type'] == 'chance'],
                'road': [space['order'] for space in data_space if space['space_type'] == 'road']
            }

            # Current space
            d_current_space = update_space(
                last_move = last_move,
                dices = dices,
                data_space = data_space
            )
            
            # Update dictionary
            template_iter['move']['before'] = last_move
            template_iter['move']['forward']['real'] = True

            # Player is on 'Go to Jail' space
            if d_current_space[0]['order'] == 31:
                # Update dictionary
                template_iter['move']['jail']['status'] = True
                template_iter['move']['after']['real'] = d_current_space[0]['order']
                template_iter['move']['after']['updated'] = 11
                template_iter['move']['forward']['real'] = True
                template_iter['move']['forward']['updated'] = False

            # Player is not on 'Go to Jail' space
            else:
                # Update dictionary
                template_iter['move']['after']['real'] = d_current_space[0]['order']
                template_iter['move']['forward']['real'] = True

                # Player is on 'Community Chest' space
                if d_current_space[0]['order'] in d_space['community_chest']:
                    # Update dictionary
                    template_iter = take_cards(
                        subiteration_dict = template_iter,
                        card = card,
                        chance_card = False,
                        data_card = data_card,
                        data_space = data_space
                    )
                if d_current_space[0]['order'] in d_space['chance']:
                    # Update dictionary
                    template_iter = take_cards(
                        subiteration_dict = template_iter,
                        card = card,
                        chance_card = True,
                        data_card = data_card,
                        data_space = data_space
                    )
                else:
                    pass
    
    return template_iter

### Simulation

In [19]:
# Sample data
sample = {
    'subiteration': 1,
    'dices': [2, 3],
    'move': {
        'chance_card': None,
        'community_card': None,
        'jail': {
            'status': False,
            'number': 0
        },
        'before': 30,
        'after': {
            'real': 35,
            'updated': None
        },
        'forward': {
            'real': None,
            'updated': None
        }
    }
}

In [20]:
# Simulation
move_spaces(
    dict_latest = sample,
    dices = [7, 6],
    subiteration = 6,
    three_times = False,
    card = True,
    data_card = data_card,
    data_space = data_space
)

{'subiteration': 6,
 'dices': [7, 6],
 'move': {'chance_card': 2,
  'community_card': None,
  'jail': {'status': False, 'number': 0},
  'before': 35,
  'after': {'real': 8, 'updated': 25},
  'forward': {'real': True, 'updated': True}}}

## Core simulation

In [21]:
# Monopoly simulation
def monopoly_simulation(
    num_player: int,
    iteration: int,
    card: bool
    ):
    # String of players
    player_str = ['Player ' + str(i) for i in range(1, num_player + 1)]
    
    # A dictionary for players
    obj = {
        'players': num_player,
        'iteration': iteration,
        'simulation': dict([(idx, dict([('iterations', [])])) for idx in player_str])
    }
    
    # Loop for iterations
    for turn in range(iteration):
        # Loop for players
        for player in player_str:
            # Length of iterations in each player
            length_iter = len(obj['simulation'][player]['iterations'])
            
            if turn == 0:
                # Initial move
                d_last_move = {
                    'subiteration': 0,
                    'dices': [0, 0],
                    'move': {
                        'chance_card': None,
                        'community_card': None,
                        'jail': {
                            'status': False,
                            'number': 0
                        },
                        'before': 1,
                        'after': {
                            'real': 1,
                            'updated': None
                        }
                    }
                }
            else:
                # Check latest move
                d_last_move = obj['simulation'][player]['iterations'][-1]['data'][-1]
            
            # CORE ALGORITHM
            # Template for subiteration
            subiteration_temp = {'iteration': turn, 'data': [d_last_move]}
            # Roll the dices
            dices = random.choices(population = range(1, 7), k = 2)
            # Subiteration
            loop = 1

            # While loop
            while (len(np.unique(dices)) == 1) and (loop < 3):
                # Get the subiteration
                subiteration_data = move_spaces(
                    dict_latest = subiteration_temp['data'][-1],
                    dices = dices,
                    subiteration = loop,
                    three_times = False,
                    card = card,
                    data_card = data_card,
                    data_space = data_space
                )

                # Append the subiteration data
                subiteration_temp['data'].append(subiteration_data)

                # Update the indexer
                dices = random.choices(population = range(1, 7), k = 2)
                loop += 1
            
            else:
                # If same numbers are found three times
                if (len(np.unique(dices)) == 1) and (loop > 2):
                    three_times = True
                # Next move
                else:
                    three_times = False
                
                # Get the subiteration
                subiteration_data = move_spaces(
                    dict_latest = subiteration_temp['data'][-1],
                    dices = dices,
                    subiteration = loop,
                    three_times = three_times,
                    card = card,
                    data_card = data_card,
                    data_space = data_space
                )

                # Append the subiteration data
                subiteration_temp['data'].append(subiteration_data)
            
            # Remove first element in data of 'subiteration_temp'
            subiteration_temp['data'] = subiteration_temp['data'][1:]
            
            # Append the obj
            obj['simulation'][player]['iterations'].append(subiteration_temp)
    
    return obj

### Convert JSON into data frame

In [22]:
# Flatten the JSON
def flatten_json(
    d: dict
    ):
    # Normalize JSON into data frame
    f = pd.json_normalize(d, sep = '_')
    # Data frame into JSON
    result_json = f.to_dict(orient = 'records')[0]
    
    return result_json

### Simulation 1 `(many players, one round)`

In [23]:
# Parameters
num_player = 5
iteration = 20000

# Simulation
data_simulation = monopoly_simulation(
    num_player = num_player,
    iteration = iteration,
    card = True
)

In [24]:
# Show the data
data_simulation

{'players': 5,
 'iteration': 20000,
 'simulation': {'Player 1': {'iterations': [{'iteration': 0,
     'data': [{'subiteration': 1,
       'dices': [6, 3],
       'move': {'chance_card': None,
        'community_card': None,
        'jail': {'status': False, 'number': 0},
        'before': 1,
        'after': {'real': 10, 'updated': None},
        'forward': {'real': True, 'updated': None}}}]},
    {'iteration': 1,
     'data': [{'subiteration': 1,
       'dices': [5, 4],
       'move': {'chance_card': None,
        'community_card': None,
        'jail': {'status': False, 'number': 0},
        'before': 10,
        'after': {'real': 19, 'updated': None},
        'forward': {'real': True, 'updated': None}}}]},
    {'iteration': 2,
     'data': [{'subiteration': 1,
       'dices': [1, 2],
       'move': {'chance_card': None,
        'community_card': None,
        'jail': {'status': False, 'number': 0},
        'before': 19,
        'after': {'real': 22, 'updated': None},
        'forwar

In [25]:
# Flatten full dictionary object
def flatten_obj(
    data_simulation: dict,
    round_iter: int
    ):
    # Players
    players = data_simulation['simulation'].keys()

    # Object to store data
    obj_simulation = []

    # Loop the players
    for player in players:
        # Number of iteration
        data_iterations = data_simulation['simulation'][player]['iterations']

        # Loop the iteration
        for iter_ in range(len(data_iterations)):
            # Selected iteration
            selected_iteration = data_iterations[iter_]
            # Iteration
            n_iteration = selected_iteration['iteration']

            # Loop the subiteration
            for subiter_ in range(len(selected_iteration['data'])):
                # Selected subiteration
                selected_subiteration = selected_iteration['data'][subiter_]
                # Flatten the JSON
                subiteration_reformat = flatten_json(selected_subiteration)
                # Add iteration, player, and round
                new_data = {**{'player': player, 'round': round_iter, 'iteration': n_iteration}, **subiteration_reformat}

                # Append into data object
                obj_simulation.append(new_data)
    
    return obj_simulation

In [26]:
# Flatten the data
obj_simulation = flatten_obj(
    data_simulation = data_simulation,
    round_iter = None
)

In [27]:
# Data frame
df = pd.DataFrame(
    data = obj_simulation
)

In [28]:
# Show the data
df.head(50)

,player,round,iteration,subiteration,dices,move_chance_card,move_community_card,move_jail_status,move_jail_number,move_before,move_after_real,move_after_updated,move_forward_real,move_forward_updated
0,Player 1,None,0,1,"[6, 3]",NaN,NaN,False,0,1,10,NaN,True,None
1,Player 1,None,1,1,"[5, 4]",NaN,NaN,False,0,10,19,NaN,True,None
2,Player 1,None,2,1,"[1, 2]",NaN,NaN,False,0,19,22,NaN,True,None
3,Player 1,None,3,1,"[4, 5]",NaN,NaN,True,0,22,31,11.0,True,False
4,Player 1,None,4,1,"[1, 6]",NaN,NaN,True,1,11,11,NaN,None,None
5,Player 1,None,5,1,"[5, 5]",NaN,NaN,False,0,11,21,NaN,True,None
6,Player 1,None,5,2,"[1, 6]",NaN,NaN,False,0,21,28,NaN,True,None
7,Player 1,None,6,1,"[1, 2]",NaN,NaN,True,0,28,31,11.0,True,False
8,Player 1,None,7,1,"[4, 5]",NaN,NaN,True,1,11,11,NaN,None,None
9,Player 1,None,8,1,"[6, 3]",NaN,NaN,True,2,11,11,NaN,None,None
